In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
for frame in frames:
    if 'num_participant' not in frame:
        frame['num_participant'] = 10000
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]

data = data[(data['num_concurrent'] == 1) | (data['operation'] == 'repair') & (data['num_concurrent'] == 2)]
data = data[data['num_participant'] == 10000]
data["outer_coding"] = list(zip(data["outer_n"], data["outer_k"]))
data["inner_coding"] = list(zip(data["inner_n"], data["inner_k"]))

def protocol_operation(row):
    return f'{row["protocol"].capitalize()} - {row["operation"]}'

def protocol_inner_coding(row):
    return f'{row["protocol"].capitalize()} - ({int(row["inner_n"])}, {int(row["inner_k"])})'

def protocol_outer_coding(row):
    return f'{row["protocol"].capitalize()} - ({int(row["outer_n"])}, {int(row["outer_k"])})'

data["Protocol - Operation"] = data.apply(protocol_operation, axis=1)
data["Protocol - Inner Code"] = data.apply(protocol_inner_coding, axis=1)
data["Protocol - Outer Code"] = data.apply(protocol_outer_coding, axis=1)
data

In [ ]:
entropy_palette = sns.cubehelix_palette(rot=0., light=0.75, n_colors=6)
entropy_palette

In [ ]:
kademlia_palette = sns.cubehelix_palette(start=2.5, rot=0., dark=0.25, n_colors=6)
kademlia_palette

In [ ]:
# hue_order = ["entropy - put", "kademlia - put", "entropy - get", "kademlia - get", "entropy - repair", "kademlia - repair"]
hue_order = [f"{p} - {c}" for c in ("(5, 4)", "(10, 8)", "(15, 12)") for p in ("Entropy", "Kademlia")]
palette = [c for p in zip(entropy_palette, reversed(kademlia_palette)) for c in p]

ax = sns.catplot(data=data[data["inner_coding"] == (80, 32)], 
    kind="bar",
    # x="outer_coding", 
    x="operation", 
    y="latency", 
    # hue="protocol - operation",
    # order=[(5, 4), (10, 8), (15, 12)],
    hue="Protocol - Outer Code",
    order=["put", "get", "repair"],
    hue_order=hue_order, palette=palette,
    # legend=False,
    height=2.56, aspect=2.)
# ax.add_legend(label_order = ["entropy - put", "entropy - get", "entropy - repair", "kademlia - put", "kademlia - get", "kademlia - repair"])
ax \
.set_xlabels("Outer Code") \
.set_xlabels("Operation") \
.set_ylabels("Latency (s)") \
.savefig("../../graphs/operation-outer.pdf")

In [ ]:
hue_order = [f"{p} - {c}" for c in ("(40, 16)", "(80, 32)", "(120, 48)") for p in ("Entropy", "Kademlia")]

sns.catplot(data=data[data["outer_coding"] == (10, 8)], 
    kind="bar",
    x="operation", y="latency", 
    # hue="protocol - operation",
    # order=[(40, 16), (80, 32), (120, 48)],
    hue="Protocol - Inner Code",
    order=["put", "get", "repair"],
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Inner Code") \
.set_xlabels("Operation") \
.set_ylabels("Latency (s)") \
.savefig("../../graphs/operation-inner.pdf")